# Lipid fingerprint analysis

This script analyzes membrane-protein or membrane-peptide MD trajectories and studies the protein-lipid interactions, including:

- Density maps --> GROMACS densmap of lipids in the XY plane

- DE index --> MDAnalysis of the depletion-enrichment index

- Occupancy --> % time a residue has been in contact w/ an specific lipid (headgroup)

- Residence time --> PyLipid analysis of theaverage residence time of a given lipid (headgroup)

- Pocket prediction --> prediction of pockets, residence times, occupancies, etc


## Import modules

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import pandas as pd
import seaborn as sns
import re
from scipy.ndimage.filters import gaussian_filter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import mdtraj
import MDAnalysis as mda
from MDAnalysis.analysis.density import DensityAnalysis
import lipyphilic
from lipyphilic.lib.area_per_lipid import AreaPerLipid
from lipyphilic.lib.memb_thickness import MembThickness
from lipyphilic.lib.assign_leaflets import AssignLeaflets
import pylipid
from pylipid.api import LipidInteraction

# Create folders

In [2]:
try:
    os.mkdir("Analysis")
    print("Folder Analysis created")
except OSError as error:
    print("Can't create folder: folder Analysis already exists")

Can't create folder: folder Analysis already exists


### 1.- Assign variables

In [3]:
traj_file = '../Sample_files/centered_step7_1.xtc' #Trajectory filename
top_file = '../Sample_files/step6.6_equilibration.gro' #Topology filename
in_memory = True #Load all trajectory to the memory?
memory_step = 10 #Memory step to load if in_memory=True

lipid_list = ["POPC", "POPS", "POSM", "POPE", "POPI"] #List of lipids to check for

contact_cutoff = 5.5 #Assign a cutoff value for the contacts 
target = "protein" #Assign target to calculate contacts

window = 10 #Window size for the sliding window averages

### 2.- Load trajectory and preparations

In [4]:
#Load trajectory as MDA universe object
u = mda.Universe(top_file, traj_file, in_memory=in_memory, in_memory_step=memory_step)

#Store times from each frame of the trajectory
times = [ts.time for ts in u.trajectory]

print("Loaded trajectory: {}".format(u))
print("Number of frames: {}".format(len(u.trajectory)))


/home/apmlab/anaconda3/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: A
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/apmlab/anaconda3/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/apmlab/anaconda3/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/apmlab/anaconda3/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: T
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(

Loaded trajectory: <Universe with 32181 atoms>
Number of frames: 1001


### 3.- Generate density maps

In [5]:
try:
    os.mkdir("Analysis/Densmaps")
    print("Folder Densmaps created")
except OSError as error:
    print("Can't create folder: folder Densmaps already exists")

Can't create folder: folder Densmaps already exists


In [6]:
for lipid in lipid_list:
    densmap = f'echo {lipid}| gmx densmap -f {traj_file} -s {top_file} -od ./Analysis/Densmaps/{lipid}_densmap.dat'
    os.system(densmap)

                      :-) GROMACS - gmx densmap, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/apmlab/Projectes/mario/Script_devel/Analysis_scripts
Command line:
  gmx densmap -f ../Sample_files/centered_step7_1.xtc -s ../Sample_files/step6.6_equilibration.gro -od ./Analysis/Densmaps/POPC_densmap.dat


Select an analysis group
Group     0 (         System) has 32181 elements
Group     1 (        Protein) has  3357 elements
Group     2 (      Protein-H) has  3357 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has     0 elements
Group     8 (      SideChain) has  3357 elements
Group     9 (    SideChain-H) has  3357 elements
Group    10 (    Prot-Masses) has  3357 elements
Group    11 (    non-Protein) has 28824 elements
Group    12 (          Ot

Selected 14: 'POPC'

  The maximum density is 1.656562 (nm^-3)


Last frame      10000 time 10000000.000   

Back Off! I just backed up ./Analysis/Densmaps/POPS_densmap.dat to ./Analysis/Densmaps/#POPS_densmap.dat.2#

GROMACS reminds you: "Beware of bugs in the above code; I have only proved it correct, not tried it." (Donald Knuth)

                      :-) GROMACS - gmx densmap, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/apmlab/Projectes/mario/Script_devel/Analysis_scripts
Command line:
  gmx densmap -f ../Sample_files/centered_step7_1.xtc -s ../Sample_files/step6.6_equilibration.gro -od ./Analysis/Densmaps/POSM_densmap.dat


Select an analysis group
Group     0 (         System) has 32181 elements
Group     1 (        Protein) has  3357 elements
Group     2 (      Protein-H) has  3357 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements

Selected 16: 'POPS'

  The maximum density is 0.461200 (nm^-3)


Last frame      10000 time 10000000.000   

Back Off! I just backed up ./Analysis/Densmaps/POSM_densmap.dat to ./Analysis/Densmaps/#POSM_densmap.dat.2#

GROMACS reminds you: "It's So Fast It's Slow" (F. Black)

                      :-) GROMACS - gmx densmap, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/apmlab/Projectes/mario/Script_devel/Analysis_scripts
Command line:
  gmx densmap -f ../Sample_files/centered_step7_1.xtc -s ../Sample_files/step6.6_equilibration.gro -od ./Analysis/Densmaps/POPE_densmap.dat


Select an analysis group
Group     0 (         System) has 32181 elements
Group     1 (        Protein) has  3357 elements
Group     2 (      Protein-H) has  3357 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has     0 elements
Group     

...

Last frame      10000 time 10000000.000   

Back Off! I just backed up ./Analysis/Densmaps/POPE_densmap.dat to ./Analysis/Densmaps/#POPE_densmap.dat.2#

GROMACS reminds you: "In the End Science Comes Down to Praying" (P. v.d. Berg)

                      :-) GROMACS - gmx densmap, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/apmlab/Projectes/mario/Script_devel/Analysis_scripts
Command line:
  gmx densmap -f ../Sample_files/centered_step7_1.xtc -s ../Sample_files/step6.6_equilibration.gro -od ./Analysis/Densmaps/POPI_densmap.dat


Select an analysis group
Group     0 (         System) has 32181 elements
Group     1 (        Protein) has  3357 elements
Group     2 (      Protein-H) has  3357 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    MainChain+H) has    

Selected 15: 'POPE'

  The maximum density is 0.950690 (nm^-3)


Reading frame    9000 time 9000000.000   

Selected 17: 'POPI'

  The maximum density is 0.893377 (nm^-3)



Reading frame   10000 time 10000000.000   
Last frame      10000 time 10000000.000   

Back Off! I just backed up ./Analysis/Densmaps/POPI_densmap.dat to ./Analysis/Densmaps/#POPI_densmap.dat.3#

GROMACS reminds you: "Occams Razor is the scientific principle that, all things being equal, the simplest explanation is always the dog ate my homework." (Greg Tamblyn)



In [5]:
#Define colors for each lipid
color_map = {"tein": "gray", "POPI": "Blues", "POPS": "Reds",
             "POSM": "Purples", "POPE": "copper_r", "POPC": "pink_r"}

#CALCULATE OVER THE MEAN [] of each lipid in the membrane?

for densmap in glob.glob("./Analysis/Densmaps/*.dat"):
    print('Plotting density map of ' + densmap[20:24])
    data = pd.read_csv(densmap, sep='\t', index_col=0)
    data = data.dropna(axis=1)

    data.index = [round(x) for x in data.index]
    data.columns = [round(float(x)) for x in data.columns]
    
    #Get min and max value of dataset
    max_val = data.max().max()
    min_val = data.min().min()
    
    #Create subplot and divide to operate heatmap and cbar separately more easily (cbar size = heatmap)
    fig, ax = plt.subplots(figsize=(5, 5))
    divider = make_axes_locatable(ax)

    cbar_ax = divider.new_vertical(size="5%", pad=0.2, pack_start=True)
    fig.add_axes(cbar_ax)
    
    #Create heatmap, put cbar in bottom, ensure it has aquare shape
    ax = sns.heatmap(data, ax=ax, cmap=color_map[densmap[-16:-12]], cbar_ax=cbar_ax, vmin=round(min_val,1), vmax=round(max_val,1),
                     cbar_kws={"orientation": "horizontal","ticks":[round(min_val,1),round(max_val,1)]}, square=True)
    ax.set(xticks=[], yticks=[])
    ax.tick_params(bottom=False, top=False)  # remove the ticks
    ax.set_aspect("equal") #Set aspect squared
    
    # Drawing the frame (border) to hea tmap
    for _, spine in ax.spines.items():
        spine.set_visible(True)
        spine.set_linewidth(3)
       
    #Get cbar and put border, its size, remove ticks, set text size    
    cbar = ax.collections[0].colorbar
    cbar.set_label("$\mathbf{nm^{-3}}$", labelpad=-13, size=23, weight="bold")
    cbar.ax.tick_params(labelsize=22, length=0, pad=10)
    cbar.ax.set_frame_on(True)
               
    for _, spine in cbar.ax.spines.items():
        spine.set_visible(True)
        spine.set_linewidth(3)                 

               
    plt.xticks(weight = 'bold') #Only cbar xticks, so we can set all bold
        
    #Save figure
    ax.get_figure()
    plt.savefig(f"{densmap[:-4]}.tiff", transparent=True, dpi=800)
    plt.clf()

Plotting density map of POPC
Plotting density map of POPS
Plotting density map of POSM
Plotting density map of POPE
Plotting density map of POPI


<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

<Figure size 360x360 with 0 Axes>

## 4.- Calculate average membrane thickness

### 4.1 Overall membrane thickness per frame

In [12]:
try:
    os.mkdir("Analysis/Memb_thickness")
    print("Folder Memb_thickness created")
except OSError as error:
    print("Can't create folder: folder Memb_thickness already exists")

Can't create folder: folder Memb_thickness already exists


In [7]:
#First assign leaflets 
leaflets = AssignLeaflets(
  universe=u,
  lipid_sel="name AM1 AM2 GL1 GL2 ROH" # assuming we are using the MARTINI forcefield
)
leaflets.run()

lipid_set_noCHOL = set(lipid_list) - set("CHOL")
print(lipid_set_noCHOL)

#Determine overall thickness per frame
memb_thickness = MembThickness(
    universe=u,
    leaflets=leaflets.filter_leaflets(f"resname {' '.join(lipid_set_noCHOL)}"),  # exclude cholesterol from thickness calculation
    lipid_sel= f"resname {' '.join(lipid_set_noCHOL)} and name PO4" #Use PO4 to PO4 distance for thickness
)

memb_thickness.run(
  start=None,
  stop=None,
  step=None,
  verbose=True
)

{'POPI', 'POPE', 'POPC', 'POPS', 'POSM'}


100%|██████████| 1001/1001 [00:01<00:00, 818.14it/s]


In [25]:
#Calculate average mbr thickness over sliding window
mbr_thickness_avg = [None for n in range(window)]
avgs = [np.mean(memb_thickness.memb_thickness[(n-window):n]) for n in range((window+1),(window + 1 + len(memb_thickness.memb_thickness[window:])))] #Calculate avgs
mbr_thickness_avg.extend(avgs)

#Convert to df
data_for_df = {'Memb_thickness':memb_thickness.memb_thickness, 'Avg_memb_thickness':mbr_thickness_avg}
df = pd.DataFrame(index=times, data=data_for_df)

#Plot membrane thickness
plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########
sns_lipid = sns.lineplot(x=df.index, y='Memb_thickness', data=df)  # Plot term over time
sns_lipid = sns.lineplot(x=df.index, y='Avg_memb_thickness', data=df) #Plot avg thickness
sns_lipid.set_xlabel('Time (ps)')
y_ax_label = 'Membrane thickness (nm)'  # get y axis name based on term being plotted
sns_lipid.set_ylabel(y_ax_label)  # Change y axis label
fig = sns_lipid.get_figure()  # Get figure
output_name = './Analysis/Memb_thickness/membrane_thickness_avg_per_frame.png'
fig.savefig(output_name)  # Save graph in folder
plt.clf()

#Save data as df
filename = './Analysis/Memb_thickness/membrane_thickness_avg_per_frame.csv'
df.to_csv(filename)

/tmp/ipykernel_911972/1658550413.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')  ####### Style for the plots (defined one to keep same style accross ALL plots ########


<Figure size 432x288 with 0 Axes>

### 4.2 Local membrane thickness calculation

[67.55252323 68.66363434 69.77474546 70.88585657 71.99696768 73.10807879
 74.2191899  75.33030101 76.44141212 77.55252323]
[66.72476239 67.55252323]


TypeError: No universe, or universe-containing object passed to the initialization of AtomGroup

In [22]:
print(dens.density)

<Density density with (167, 165, 72) bins>


/home/apmlab/anaconda3/lib/python3.9/site-packages/MDAnalysis/analysis/density.py:497: DeprecationWarning: The `density` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.density` instead
  warnings.warn(wmsg, DeprecationWarning)


## 5.- Calculate APL

In [30]:
upper_areas = AreaPerLipid(
  universe=u,
  lipid_sel="name AM1 AM2 GL1 GL2 ROH",  # assuming we're using the MARTINI forcefield
  leaflets= leaflets.leaflets
)

areas.run(start=None, stop=None, step=None, verbose=True)
areas.areas

  0%|          | 0/1001 [00:00<?, ?it/s]

array([[ 78.08574486,  89.56125541,  71.78617098, ...,  46.10061399,
         65.40614736,  60.7730861 ],
       [ 66.52627686, 102.20163254,  81.51327871, ...,  53.04753003,
         80.48184413,  59.2036968 ],
       [ 49.35191802, 174.59212471, 159.1161145 , ...,  42.61175425,
         83.34926635,  58.99578696],
       ...,
       [ 64.5987028 ,  52.97322205,  57.1407385 , ...,  52.0418672 ,
         80.96358182,  54.1673949 ],
       [ 34.41319683,  68.58039907,  52.23826105, ...,  43.02451654,
         60.55287738,  78.73146005],
       [ 83.65989206,  57.30486995,  54.6368958 , ...,  73.37532885,
         61.52091511,  76.1537314 ]])

In [32]:
areas.project_area()

NoDataError: AtomGroup.unwrap() not available; this requires Bonds

## 6.- Calculate lipid order parameter

In [24]:
#Calculate order lipid parameters
scc_sn1 = SCC(
  universe=u,
  tail_sel="name ??A"  # selects C1A, C2A, D2A, C3A, and C4A
)

scc_sn2 = SCC(
  universe=u,
  tail_sel="name ??B"  # selects C1B, C2B, D2B, C3B, and C4B
)
scc_sn2.run(verbose=True)
scc_sn1.run(verbose=True)

#Calculaye avg between the 2 tails: returns array of size n_lipids x n_frames
scc_avg = SCC.weighted_average(scc_sn1, scc_sn2)

print(scc_avg.SCC)

  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

[[ 0.15015222  0.52005266  0.45644873 ...  0.38368262  0.03976182
   0.19064553]
 [ 0.44860431  0.3521393   0.46379279 ...  0.28350177  0.1277843
   0.31811381]
 [ 0.43166041  0.53183657  0.20009969 ...  0.26218559  0.14151495
   0.41575104]
 ...
 [ 0.20050992  0.41296375  0.37887865 ...  0.27207282  0.44955107
   0.21194587]
 [ 0.25427989  0.35253109 -0.05964335 ...  0.70321646  0.45138725
   0.23980423]
 [ 0.12728618  0.36455576 -0.25911421 ...  0.52096539  0.23516731
   0.5806046 ]]


## 7. Pylipid: calculate residue occupancy

In [39]:
try:
    os.mkdir("Analysis/PyLipid")
    print("Folder Pylipid created")
except OSError as error:
    print("Can't create folder: folder Pylipid already exists")

Folder Pylipid created


In [48]:
#Loop for every lipid

#Prepare data for PyLipid module
cutoffs = [0.475, 0.8] #Must be optimized for each lipid
nprot=1
timeunit = "us"
save_dir = "./Analysis/PyLipid/"



#Initialize
li = pylipid.api.LipidInteraction(traj_file, topfile_list=top_file, cutoffs=cutoffs, lipid="POPI",
                      nprot=nprot, save_dir=save_dir)

Creating new director: /home/apmlab/Projectes/mario/Script_devel/Analysis_scripts/Analysis/PyLipid/Interaction_POPI


In [51]:
li.collect_residue_contacts()
durations = li.compute_residue_duration()
occupancies = li.compute_residue_occupancy()
lipidcounts = li.compute_residue_lipidcount()

CALCULATE RESIDUE LIPIDCOUNT: 100%|██████████| 1482/1482 [00:01<00:00, 1035.01it/s]


In [63]:
li.dataset[li.dataset["Occupancy"] > 20]

,Residue,Residue ID,Duration,Duration std,Occupancy,Occupancy std,Lipid Count,Lipid Count std
38,39PHE,38,0.017278,0.028929,34.606539,0.0,1.133487,0.0
40,41PHE,40,0.053278,0.205977,20.877912,0.0,1.021552,0.0
42,43PHE,42,0.014996,0.024983,22.117788,0.0,1.067812,0.0
49,50PHE,49,0.013972,0.025623,21.517848,0.0,1.058086,0.0
50,51HIS,50,0.116184,0.342478,25.497450,0.0,1.038824,0.0
55,56VAL,55,0.057868,0.111415,20.067993,0.0,1.017439,0.0
130,131PHE,130,0.036855,0.112024,26.247375,0.0,1.116190,0.0
132,133LYS,132,0.051147,0.115830,25.687431,0.0,1.095368,0.0
133,134PHE,133,0.071647,0.188060,29.787021,0.0,1.114804,0.0
135,136ARG,135,0.040076,0.151729,31.036896,0.0,1.094394,0.0
